# Otimização das tabelas da camada Gold

## Carregando as bibliotecas

In [6]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import SparkSession


StatementMeta(, 0b8103b5-d08d-4089-bc53-b79312076286, 8, Finished, Available, Finished)

## SPARK SESSION

In [7]:
spark = SparkSession.builder.appName("TransformationsGoldLayerFacts").getOrCreate()

StatementMeta(, 0b8103b5-d08d-4089-bc53-b79312076286, 9, Finished, Available, Finished)

## Importação dos dados da camada Silver
Aqui temos algumas opções: 
- Importar do storage do Data Lake (landing zone);
- Importar de um shortcut;
- Importar da Tabela (Table) propagada do Lakehouse no schema broze;

Nos exemplos abaixo usaremos a terceira opção por simplicidade. 

In [8]:
# Usando Variaveis para parametrizar os codigos:
INPUT_LAYER = "gold"
WORKSPACE_NAME = spark.conf.get("trident.workspace.name")
LAKEHOUSE_NAME = spark.conf.get("trident.lakehouse.name")
OUTPUT_LAYER = "gold"

# Diretorio de saida completo
INPUT_PATH = f"abfss://{WORKSPACE_NAME}@onelake.dfs.fabric.microsoft.com/{LAKEHOUSE_NAME}.Lakehouse/Tables/{INPUT_LAYER}"
OUTPUT_PATH = f"abfss://{WORKSPACE_NAME}@onelake.dfs.fabric.microsoft.com/{LAKEHOUSE_NAME}.Lakehouse/Tables/{OUTPUT_LAYER}"
# Diretorio de saida curto
# Tables/{OUTPUT_LAYER}/olist_customers_dataset.csv

StatementMeta(, 0b8103b5-d08d-4089-bc53-b79312076286, 10, Finished, Available, Finished)

In [9]:
fact_sales_df = spark.read.format("delta")\
        .load(f"{INPUT_PATH}/fact_sales")


StatementMeta(, 0b8103b5-d08d-4089-bc53-b79312076286, 11, Finished, Available, Finished)

# Otimização da camada Gold para melhor performance de Query

## Objetivo
Agora que criamos todas as Tabelas de Dimensão, Tabelas de Fatos, Tabelas de Agregação de Fatos e Tabelas de Ponte, precisamos otimizar a Camada Gold para consultas rápidas e armazenamento eficiente.

### Otimizações Aplicadas
- **Particionamento** → Melhora o desempenho das consultas organizando os dados em partições
- **Z-Ordering (Clustering)** → Ordena os dados eficientemente dentro das partições para reduzir I/O
- **Vacuuming de Tabelas Delta** → Remove versões antigas e otimiza o armazenamento
- **Otimizações de Desempenho de Tabelas Delta** → Habilita a omissão de dados e indexação

**Particionamento das Tabelas da Camada Gold**  
O particionamento divide grandes tabelas em partes menores e mais gerenciáveis. Isso reduz os tempos de varredura e melhora o desempenho.  



In [14]:
# Particionando a tabela FactSales por Order Year

fact_sales_df.write \
    .partitionBy("purchase_date") \
    .format("delta") \
    .mode("overwrite") \
    .option("path", f"{OUTPUT_PATH}/fact_sales_partitioned")\
    .save()

# Particionando a tabela FactOrderPayments por Payment Type
# fact_order_payments_df = spark.read.format("delta").load(f"{OUTPUT_PATH}/fact_sales_partitioned")
fact_order_payments_df = fact_sales_df
fact_order_payments_df.write \
    .partitionBy("payment_type") \
    .format("delta") \
    .mode("overwrite") \
    .option("path", f"{OUTPUT_PATH}/fact_order_payments_partitioned")\
    .save()


StatementMeta(, 0b8103b5-d08d-4089-bc53-b79312076286, 16, Finished, Available, Finished)

## Implementação do Z-Ordering para consultas mais rápidas


**Z-Ordering** é uma técnica que organiza os dados com base em colunas frequentemente consultadas, o que reduz a necessidade de varredura de arquivos e melhora ainda mais a performance. Isso é especialmente útil em grandes conjuntos de dados onde a otimização de I/O pode fazer uma grande diferença.



In [15]:
from delta.tables import DeltaTable

# Otimizando a tabela FactSales com Z-Ordering na coluna Customer SK
fact_sales_table = DeltaTable.forPath(spark, f"{OUTPUT_PATH}/fact_sales_partitioned")

fact_sales_table.optimize().executeZOrderBy("fact_sales_sk")

# Otimizando a tabela DimCustomer com Z-Ordering na coluna City
dim_customer_table = DeltaTable.forPath(spark, f"{OUTPUT_PATH}/dim_customer")

dim_customer_table.optimize().executeZOrderBy("customer_city")

StatementMeta(, 0b8103b5-d08d-4089-bc53-b79312076286, 17, Submitted, Running, Running)

## Usando o Vacuum nas tabelas Delta

As tabelas Delta armazenam versões históricas dos dados, permitindo viagens no tempo e rollback. Para liberar espaço de armazenamento, podemos usar o **vacuum** para remover dados antigos.


In [ ]:
# Vacuum para remover as versões antigas da FactSales
fact_sales_table.vacuum(retentionHours=168)  # Keep last 7 days

# Vacuum na tabela DimCustomer 
dim_customer_table.vacuum(retentionHours=168)


StatementMeta(, , -1, Waiting, , Waiting)